In [44]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from pandas.core.frame import DataFrame

In [37]:
cri = pd.read_csv("df_train_val06.csv")
test_df = pd.read_csv("df_test.csv")

cri

,yday,time,vic_age_group,vic_race,vic_sex,service,cluster
0,209,16,<18,BLACK,F,7 Avenue(123),Middle Manhattan
1,60,8,<18,WHITE,F,7 Avenue(123),Middle Manhattan
2,323,8,18-24,ASIAN / PACIFIC ISLANDER,M,7 Avenue(123),Middle Manhattan
3,355,8,<18,WHITE,M,7 Avenue(123),Middle Manhattan
4,42,20,25-44,WHITE,F,7 Avenue(123),Middle Manhattan
...,...,...,...,...,...,...,...
77528,182,12,18-24,WHITE,M,Shuttle(S),Queen-Brooklyn
77529,2,0,18-24,BLACK,F,Shuttle(S),Queen-Brooklyn
77530,110,8,25-44,WHITE HISPANIC,F,Shuttle(S),Queen-Brooklyn
77531,199,0,18-24,BLACK,F,Shuttle(S),Queen-Brooklyn


In [38]:
cri["user_id"] = cri['yday'].map(str) + "_" + cri['time'].map(str) 
cri["item_id"] = cri['vic_age_group'].map(str) + cri['vic_race'].map(str) + cri['vic_sex'].map(str) + cri['service'].map(str)+ '_' + cri['cluster'].map(str)
test_df["user_id"] = test_df['yday'].map(str) + "_" + test_df['time'].map(str) 
test_df["item_id"] = test_df['vic_age_group'].map(str) + test_df['vic_race'].map(str) + test_df['vic_sex'].map(str) + test_df['service'].map(str)+ '_' + test_df['cluster'].map(str)

In [40]:

le = preprocessing.LabelEncoder()
for column in test_df[['user_id','item_id']]:
    le.fit(cri[column])
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    cri[column] = le.transform(cri[column])
    test_df[column] = test_df[column].apply(lambda x: le_dict.get(x, -9000))
cri['item_id']= cri['item_id']+ cri.max()['user_id'] + 1   
test_df['item_id']= test_df['item_id']+ cri.max()['user_id'] + 1   
for column in cri[['user_id','item_id']]:
    test_df = test_df[test_df[column].isin(list(cri[column].unique()))]

In [55]:
#training data
#cri[['user_id','item_id']].sort_values(by=['user_id']).to_csv('onewaydata.csv')

In [216]:
test_df

,yday,time,vic_age_group,vic_race,vic_sex,service,cluster,user_id,item_id
2,145,0,18-24,WHITE,F,7 Avenue(123),Middle Manhattan,306,2524
3,267,4,45-64,BLACK,M,7 Avenue(123),Middle Manhattan,1120,3197


In [217]:
z_np = np.loadtxt('zz_np', dtype=np.float32, delimiter=',')
pred = np.sum(z_np[test_df['user_id']] * z_np[test_df['item_id']],axis=1)
pred

array([0.8334638 , 0.79700243], dtype=float32)